In [1]:
import cv2
import os
import keras
import numpy as np
import matplotlib as plt
from keras.models import Sequential, Model
from keras.layers import Dense , Conv2D, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D, Input, Conv2DTranspose, Input
from keras.utils import np_utils
from keras.optimizers import Adadelta, RMSprop,SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split

seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [3]:
d = 'G:\Study\Project\ALL\ALL_IDB\A\im'
d=os.listdir(d)
d

['Im001_1.jpg', 'Im002_1.jpg', 'Im003_1.jpg', 'Im004_1.jpg']

In [8]:
def data_loader(data):
    train_list0=os.listdir(data)
    # Map class names to integer labels
    # train_class_labels = { label: index for index, label in enumerate(class_names) } 
      
    # Number of classes in the dataset
    #num_classes=len(train_list0)

    # Empty lists for loading training and testing data images as well as corresponding labels
    x=[]
    
    # Loading training data
    path1=data                        #+'/'+str(elem)
    images=os.listdir(path1)
    for elem2 in images:
        path2=path1+'/'+str(elem2)
        # Read the image form the directory
        img = cv2.imread(path2)  
        # Append image to the train data list
        x.append(img)
        
   
    # Convert lists into numpy arrays

    x = np.asarray(x)

    return x

In [9]:
d = 'G:\Study\Project\ALL\ALL_IDB\A\im'

In [11]:
X = data_loader(d)

input_shape = (X.shape[1], X.shape[2], X.shape[3])

X = X.astype('float32')

X = X / 255.

In [12]:
###########################################  Encoder  ####################################################


def Encoder(input_img):

	Econv1_1 = Conv2D(16, (3, 3), activation='relu', padding='same', name = "block1_conv1")(input_img)
	Econv1_1 = BatchNormalization()(Econv1_1)
	Econv1_2 = Conv2D(16, (3, 3), activation='relu', padding='same',  name = "block1_conv2")(Econv1_1)
	Econv1_2 = BatchNormalization()(Econv1_2)
	pool1 = MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding='same', name = "block1_pool1")(Econv1_2)
	
	Econv2_1 = Conv2D(64, (3, 3), activation='relu', padding='same', name = "block2_conv1")(pool1)
	Econv2_1 = BatchNormalization()(Econv2_1)
	Econv2_2 = Conv2D(64, (3, 3), activation='relu', padding='same', name = "block2_conv2")(Econv2_1)
	Econv2_2 = BatchNormalization()(Econv2_2)
	pool2= MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same', name = "block2_pool1")(Econv2_2)

	Econv3_1 = Conv2D(128, (3, 3), activation='relu', padding='same', name = "block3_conv1")(pool2)
	Econv3_1 = BatchNormalization()(Econv3_1)
	Econv3_2 = Conv2D(128, (3, 3), activation='relu', padding='same', name = "block3_conv2")(Econv3_1)
	Econv3_2 = BatchNormalization()(Econv3_2)
	pool3 = MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same', name = "block3_pool1")(Econv3_2)

	encoded = Model(inputs = input_img, outputs = pool3 )

	return encoded

In [13]:
##########################################  Decoder   ##################################################

def Decoder(inp ):

	up1 = Conv2DTranspose(128,(3,3),strides = (2,2), activation = 'relu', padding = 'same', name = "upsample_1")(inp)
	up1 = BatchNormalization()(up1)
	#up1 = merge([up1, inp[3]], mode='concat', concat_axis=3, name = "merge_1")
	Upconv1_1 = Conv2D(128, (3, 3), activation='relu', padding='same', name = "Upconv1_1")(up1)
	Upconv1_1 = BatchNormalization()(Upconv1_1)
	Upconv1_2 = Conv2D(128, (3, 3), activation='relu', padding='same', name = "Upconv1_2")(Upconv1_1)
	Upconv1_2 = BatchNormalization()(Upconv1_2)

	up2 = Conv2DTranspose(64,(3,3),strides = (2,2), activation = 'relu', padding = 'same', name = "upsample_2")(Upconv1_2)
	up2 = BatchNormalization()(up2)
	#up2 = merge([up2, inp[2]], mode='concat', concat_axis=3, name = "merge_2")
	Upconv2_1 = Conv2D(64, (3, 3), activation='relu', padding='same', name = "Upconv2_1")(up2)
	Upconv2_1 = BatchNormalization()(Upconv2_1)
	Upconv2_2 = Conv2D(64, (3, 3), activation='relu', padding='same', name = "Upconv2_2")(Upconv2_1)
	Upconv2_2 = BatchNormalization()(Upconv2_2)
	
	up3 = Conv2DTranspose(16,(3,3),strides = (2,2), activation = 'relu', padding = 'same', name = "upsample_3")(Upconv2_2)
	up3 = BatchNormalization()(up3)
	#up3 = merge([up3, inp[1]], mode='concat', concat_axis=3, name = "merge_3")
	Upconv3_1 = Conv2D(16, (3, 3), activation='relu', padding='same', name = "Upconv3_1")(up3)
	Upconv3_1 = BatchNormalization()(Upconv3_1)
	Upconv3_2 = Conv2D(16, (3, 3), activation='relu', padding='same', name = "Upconv3_2")(Upconv3_1)
	Upconv3_2 = BatchNormalization()(Upconv3_2)
	   
	decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same', name = "Ouput_layer")(Upconv3_2)
	convnet = Model(inputs = inp, outputs =  decoded)

	return convnet

In [ ]:
input_img = Input(shape=input_shape)

#Encoders[1]*100))
encoded = Encoder(input_img)	#return encoded representation with intermediate layer Pool3(encoded)

#Decoder
decoded = Decoder(Input(shape=(214, 171, 128)))

output_img = decoded(encoded(input_img))

model= Model(inputs = input_img, outputs = output_img )

model.summary()

model.compile(optimizer = SGD(0.0002), loss='binary_crossentropy', metrics = ["accuracy"])

model.fit(X, X, batch_size=1, epochs=5, verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1368, 1712, 3)     0         
_________________________________________________________________
model_1 (Model)              (None, 171, 214, 128)     272080    
_________________________________________________________________
model_2 (Model)              multiple                  607203    
Total params: 879,283
Trainable params: 877,203
Non-trainable params: 2,080
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
